# Long Short-Term Memory (LSTM)

## LSTM is a type of RNN that can remember past values over long periods of time. It is useful for predicting stock prices that have long-term dependencies.

1. Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2. Load the data

In [ ]:
df = pd.read_csv('commbank.csv')

3. Preprocess the data

In [ ]:
# Set 'Date' column as index
df.set_index('Date', inplace=True)

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Split the data into training and testing sets
train_data = scaled_data[:int(0.8 * len(df)), :]
test_data = scaled_data[int(0.8 * len(df)):, :]

4. Create data sequences for training and testing

In [ ]:
# Define function to create data sequences
def create_sequences(data, seq_length):
    x = []
    y = []
    for i in range(seq_length, len(data)):
        x.append(data[i-seq_length:i, :])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

# Create sequences for training and testing
seq_length = 60
train_x, train_y = create_sequences(train_data, seq_length)
test_x, test_y = create_sequences(test_data, seq_length)

5. Build the LSTM model 

In [ ]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])),
    LSTM(units=50, return_sequences=True),
    LSTM(units=50),
    Dense(units=1)
])

6. Compile the model

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

7. Train the model

In [ ]:
history = model.fit(train_x, train_y, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

8. Evaluate the model on the test set

In [ ]:
# Get the predicted closing prices
predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Get the actual closing prices
actuals = scaler.inverse_transform(test_y.reshape(-1, 1))

# Calculate evaluation metrics
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAE: ', mae)
print('R2: ', r2)

9. Visualize the results

In [ ]:
# Plot actual vs predicted closing prices
plt.figure(figsize=(16,8))
plt.plot(actuals, color='blue', label='Actual')
plt.plot(predictions, color='red', label='Predicted')
plt.title('Actual vs Predicted Closing Prices')
plt.xlabel('Time')
plt.ylabel('Closing Price')
plt.legend()
plt.show()

# Plot training and validation loss
plt.figure(figsize=(16,8))
plt.plot(history.history['loss'], color='blue', label='Training Loss')
plt.plot(history.history['val_loss'], color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()